In [80]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_keys

#Configure gmaps
gmaps.configure(api_key=g_keys)

In [56]:
vacation_df= pd.read_csv("weather_json.csv", index_col=0)
vacation_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudness,WindSpeed,Country,Date
0,luderitz,-26.6481,15.1594,69.66,28,0,15.99,NaN,1657950535
1,broken hill,-31.9500,141.4333,60.39,20,0,24.11,AU,1657950640
2,kapaa,22.0752,-159.3190,82.38,80,40,14.97,US,1657950506
3,bethel,41.3712,-73.4140,69.66,85,0,0.00,US,1657950530
4,dikson,73.5069,80.5464,41.05,89,62,7.54,RU,1657950510


In [57]:
# Finding the locations by using the coordinates 
locations = vacation_df[["Lat", "Lng"]]

humidity = vacation_df["Humidity"]

In [60]:
# Plotting Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Adding layers
fig.add_layer(heat_layer)


# Displaying and saving map
plt.savefig('city_heatmap.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [61]:
# Filtering DataFrame to find ideal weather condition for vacation

ideal_weather_df= vacation_df.loc[(vacation_df["Max_Temp"] < 80) & (vacation_df["Max_Temp"] > 70) & (vacation_df["WindSpeed"] < 10) & (vacation_df["Cloudness"] == 0)]

In [1]:
hotel_df = ideal_weather_df.reset_index(drop=True)


NameError: name 'ideal_weather_df' is not defined

In [84]:
# Seting variable
hotels = []

# Loop through filtered dataframe to get nearby hotel
for i, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    target_coordinates = f"{lat}, {lng}"
    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates, 
        "radius": target_radius,
        "types": target_type,
        "key": g_keys
         }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    response = requests.get(base_url, params=params)  
    response = response.json()
    try:

        hotel_df.loc[i,"Hotel"]=response["results"][0]["name"]
        
    except(IndexError):
        print("Hotel not found")
        
        
    
# Dataframe with nearest hotel
#hotel_df["Hotel"] = hotels
#hotel_df

Hotel not found


In [87]:
hotel_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudness,WindSpeed,Country,Date,Hotel
0,laguna,38.4210,-121.4238,79.93,48,0,6.91,US,1657950579,Holiday Inn Express & Suites Elk Grove Central...
1,manaus,-3.1019,-60.0250,79.29,83,0,2.30,BR,1657950647,Manaus Hotéis Millennium
2,pombia,45.6530,8.6336,78.22,68,0,3.44,IT,1657950648,Albergo Fontana
3,misratah,32.3754,15.0925,77.63,57,0,1.48,LY,1657950488,Almasa Hotel
4,drumheller,51.4617,-112.7101,71.44,56,0,3.44,CA,1657950656,Ramada by Wyndham Drumheller Hotel & Suites


In [76]:
#for city in range(len(hotel_df["City"])):
   # print(city)

In [77]:
#hotel_df.loc[0,'Lat']

In [91]:
hotels = hotel_df["Hotel"].tolist()

In [92]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Name: {hotel}" for hotel in hotels])
                            
fig.add_layer(markers)

# Displaying and saving Map 
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>